In [14]:
!pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 OPR/118.0.0.0"
}
url = "https://finance.yahoo.com/quote/AAPL/"

In [4]:
r = requests.get(url , headers=headers)

In [5]:
print(r.status_code)

200


In [6]:
soup= BeautifulSoup(r.text,"html.parser")

In [7]:
def get_percent_change(url, time_range):
    span_tags = soup.select("span.container.tw-px-3.yf-15g2hux")
    for span_tag in span_tags:
        text = span_tag.get_text(strip=True)
        if text.startswith(time_range):
            return text[len(time_range):]
    return None


url = "https://finance.yahoo.com/quote/AAPL/"
print(get_percent_change(url, "5D"))
print(get_percent_change(url, "1M"))
print(get_percent_change(url, "6M"))
print(get_percent_change(url, "1Y"))





-1.42%
-5.68%
-16.06%
4.69%


In [8]:
import time

def get_percent_change(soup, time_range):
    span_tags = soup.select("span.container.tw-px-3.yf-15g2hux")
    for span_tag in span_tags:
        text = span_tag.get_text(strip=True)
        if text.startswith(time_range):
            return text[len(time_range):]
    return None

def fetch_momentum(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    changes = []
    for period in ["5D", "1M", "6M", "1Y"]:
        val = get_percent_change(soup, period)
        if val is None:
            return None  
        val = float(val.replace("%", "").replace(",", ""))
        changes.append(val)

    momentum = sum(changes) / len(changes)
    return momentum

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN"]

momentum_scores = {}

for ticker in tickers:
    print(f"Checking: {ticker}")
    momentum = fetch_momentum(ticker)
    if momentum is not None:
        momentum_scores[ticker] = momentum
    else:
        print(f"  ➤ {ticker} için veri eksik.")
    time.sleep(2)

print("\n🔹 Momentum Skorları:")
for ticker, score in momentum_scores.items():
    print(f"{ticker}: {score:.2f}")


Checking: AAPL
Checking: MSFT
Checking: GOOGL
Checking: AMZN
  ➤ AMZN için veri eksik.

🔹 Momentum Skorları:
AAPL: -4.68
MSFT: 7.69
GOOGL: 1.67


In [9]:
momentum_scores

{'AAPL': -4.677499999999999, 'MSFT': 7.6925, 'GOOGL': 1.665}

In [10]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)

df = tables[0]

tickers = df['Symbol'].tolist()

pd.Series(tickers).to_csv('sp500_tickers.csv', index=False)

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

df = pd.read_csv(r"C:\Users\Woodpecker\momemntum\sp500_tickers.csv")

def get_percent_change(ticker, time_range):
    url = f"https://finance.yahoo.com/quote/{ticker}/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, "html.parser")
        span_tags = soup.select("span.container.tw-px-3.yf-15g2hux")
        for span_tag in span_tags:
            text = span_tag.get_text(strip=True)
            if text.startswith(time_range):
                return text[len(time_range):]
        return None
    except Exception as e:
        print(f"Hata: {ticker} - {e}")
        return None

momentum_scores = {}

for i, ticker in enumerate(tickers):
    try:
        changes = []
        for t in ["5D", "1M", "6M", "1Y"]:
            change = get_percent_change(ticker, t)
            if change:
                change = float(change.replace('%', '').replace(',', ''))
                changes.append(change)
            else:
                changes.append(0.0)
            time.sleep(1)  
        
        momentum_scores[ticker] = sum(changes)

        print(f"{i+1}/{len(tickers)} {ticker} | Momentum: {momentum_scores[ticker]:.2f}")

    except Exception as e:
        print(f"Hata oluştu: {ticker} - {e}")



1/503 MMM | Momentum: 69.88
2/503 AOS | Momentum: -43.45
3/503 ABT | Momentum: 41.38
4/503 ABBV | Momentum: 17.03
5/503 ACN | Momentum: -1.27
6/503 ADBE | Momentum: -24.59
7/503 AMD | Momentum: -13.50
8/503 AES | Momentum: -81.34
9/503 AFL | Momentum: -0.90
10/503 A | Momentum: -30.02
11/503 APD | Momentum: -17.42
12/503 ABNB | Momentum: -18.58
13/503 AKAM | Momentum: -44.02
14/503 ALB | Momentum: -104.65
15/503 ARE | Momentum: -74.77
16/503 ALGN | Momentum: -43.37
17/503 ALLE | Momentum: 24.18
18/503 LNT | Momentum: -3.68
19/503 ALL | Momentum: 26.14
20/503 GOOGL | Momentum: 6.27
21/503 GOOG | Momentum: 5.21
22/503 MO | Momentum: 2.26
23/503 AMZN | Momentum: 22.08
24/503 AMCR | Momentum: -28.67
25/503 AEE | Momentum: 32.72
26/503 AEP | Momentum: 12.07
27/503 AXP | Momentum: 32.86
28/503 AIG | Momentum: 16.02
29/503 AMT | Momentum: 9.98
30/503 AWK | Momentum: 11.34
31/503 AMP | Momentum: 14.17
32/503 AME | Momentum: 6.67
33/503 AMGN | Momentum: -4.26
34/503 APH | Momentum: 73.88
35/503

In [13]:
 momentum_scores

{'MMM': 69.88,
 'AOS': -43.449999999999996,
 'ABT': 41.379999999999995,
 'ABBV': 17.03,
 'ACN': -1.2699999999999987,
 'ADBE': -24.59,
 'AMD': -13.499999999999996,
 'AES': -81.34,
 'AFL': -0.9000000000000021,
 'A': -30.02,
 'APD': -17.42,
 'ABNB': -18.58,
 'AKAM': -44.019999999999996,
 'ALB': -104.65,
 'ARE': -74.77000000000001,
 'ALGN': -43.370000000000005,
 'ALLE': 24.18,
 'LNT': -3.68,
 'ALL': 26.14,
 'GOOGL': 6.27,
 'GOOG': 5.210000000000001,
 'MO': 2.26,
 'AMZN': 22.08,
 'AMCR': -28.67,
 'AEE': 32.72,
 'AEP': 12.070000000000002,
 'AXP': 32.86,
 'AIG': 16.02,
 'AMT': 9.98,
 'AWK': 11.34,
 'AMP': 14.170000000000002,
 'AME': 6.670000000000001,
 'AMGN': -4.26,
 'APH': 73.88,
 'ADI': 0.3099999999999996,
 'ANSS': -5.539999999999999,
 'AON': 29.950000000000003,
 'APA': -58.15,
 'APO': -14.269999999999998,
 'AAPL': -23.53,
 'AMAT': -29.49,
 'APTV': 22.9,
 'ACGL': -14.6,
 'ADM': -31.68,
 'ANET': 5.3100000000000005,
 'AJG': 53.43000000000001,
 'AIZ': -8.309999999999999,
 'T': 77.29,
 'ATO': 

In [14]:
df = pd.DataFrame.from_dict(momentum_scores, orient='index')
df

,0
MMM,69.88
AOS,-43.45
ABT,41.38
ABBV,17.03
ACN,-1.27
...,...
XYL,-5.02
YUM,5.12
ZBRA,-23.09
ZBH,-47.10


In [15]:
dfs = df.sort_values(by=0, ascending=False)
dfs[0] = dfs[0] / 4
dfs

,0
PLTR,145.8450
GEV,61.2600
AXON,49.5400
NRG,44.1575
HWM,43.5100
...,...
DECK,-25.0925
ALB,-26.1625
MRNA,-29.5750
UNH,-29.9000


In [53]:
dfs50=dfs[:50].copy()

In [54]:
yatirim = float(input("Toplam yatırım miktarını gir: "))
pay = yatirim / 50

Toplam yatırım miktarını gir:  100000


In [55]:
dfs50["Yatırım Tutarı"] = pay


In [56]:
dfs50

,0,Yatırım Tutarı
PLTR,145.8450,2000.0
GEV,61.2600,2000.0
AXON,49.5400,2000.0
NRG,44.1575,2000.0
HWM,43.5100,2000.0
AVGO,38.3800,2000.0
TSLA,35.4275,2000.0
PODD,34.5125,2000.0
TPR,31.4375,2000.0
NFLX,29.9550,2000.0


In [29]:
liste = dfs50.index.tolist()

In [30]:
liste

['PLTR',
 'GEV',
 'AXON',
 'NRG',
 'HWM',
 'AVGO',
 'TSLA',
 'PODD',
 'TPR',
 'NFLX',
 'PM',
 'GE',
 'DASH',
 'RL',
 'GILD',
 'RCL',
 'STX',
 'CAH',
 'CEG',
 'JCI',
 'VRSN',
 'VST',
 'FOXA',
 'FFIV',
 'PAYC',
 'JBL',
 'CRWD',
 'INTU',
 'DRI',
 'FOX',
 'T',
 'MOS',
 'FTNT',
 'APH',
 'BK',
 'MMM',
 'BA',
 'IBM',
 'META',
 'EQT',
 'LUV',
 'WELL',
 'CME',
 'BKNG',
 'EBAY',
 'TT',
 'WRB',
 'WMT',
 'RSG',
 'TTWO']

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_price_yahoo(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        price_span = soup.find("span", {"data-testid": "qsp-price"})
        if price_span:
            return float(price_span.text.replace(",", "").strip())
        else:
            return None
    except Exception as e:
        print(f"{ticker} için hata: {e}")
        return None




In [37]:
fiyat_dict = {}
for i, ticker in enumerate(liste):
    fiyat = get_price_yahoo(ticker)
    fiyat_dict[ticker] = fiyat
    print(f"{i+1}/{len(liste)}: {ticker} fiyatı → {fiyat}")
    time.sleep(1.2)  

1/50: PLTR fiyatı → 123.01
2/50: GEV fiyatı → 474.43
3/50: AXON fiyatı → 744.96
4/50: NRG fiyatı → 153.82
5/50: HWM fiyatı → 170.09
6/50: AVGO fiyatı → 240.83
7/50: TSLA fiyatı → 360.89
8/50: PODD fiyatı → 323.89
9/50: TPR fiyatı → 78.42
10/50: NFLX fiyatı → 1187.02
11/50: PM fiyatı → 177.43
12/50: GE fiyatı → 243.64
13/50: DASH fiyatı → 205.0
14/50: RL fiyatı → 280.11
15/50: GILD fiyatı → 111.04
16/50: RCL fiyatı → 253.46
17/50: STX fiyatı → 117.93
18/50: CAH fiyatı → 153.73
19/50: CEG fiyatı → 304.26
20/50: JCI fiyatı → 101.91
21/50: VRSN fiyatı → 267.45
22/50: VST fiyatı → 160.82
23/50: FOXA fiyatı → 54.27
24/50: FFIV fiyatı → 280.6
25/50: PAYC fiyatı → 256.43
26/50: JBL fiyatı → 168.15
27/50: CRWD fiyatı → 461.71
28/50: INTU fiyatı → 754.38
29/50: DRI fiyatı → 213.63
30/50: FOX fiyatı → 49.73
31/50: T fiyatı → 27.41
32/50: MOS fiyatı → 35.99
33/50: FTNT fiyatı → 101.43
34/50: APH fiyatı → 88.89
35/50: BK fiyatı → 89.23
36/50: MMM fiyatı → 149.38
37/50: BA fiyatı → 206.65
38/50: IBM

In [38]:
fiyat_dict

{'PLTR': 123.01,
 'GEV': 474.43,
 'AXON': 744.96,
 'NRG': 153.82,
 'HWM': 170.09,
 'AVGO': 240.83,
 'TSLA': 360.89,
 'PODD': 323.89,
 'TPR': 78.42,
 'NFLX': 1187.02,
 'PM': 177.43,
 'GE': 243.64,
 'DASH': 205.0,
 'RL': 280.11,
 'GILD': 111.04,
 'RCL': 253.46,
 'STX': 117.93,
 'CAH': 153.73,
 'CEG': 304.26,
 'JCI': 101.91,
 'VRSN': 267.45,
 'VST': 160.82,
 'FOXA': 54.27,
 'FFIV': 280.6,
 'PAYC': 256.43,
 'JBL': 168.15,
 'CRWD': 461.71,
 'INTU': 754.38,
 'DRI': 213.63,
 'FOX': 49.73,
 'T': 27.41,
 'MOS': 35.99,
 'FTNT': 101.43,
 'APH': 88.89,
 'BK': 89.23,
 'MMM': 149.38,
 'BA': 206.65,
 'IBM': 258.92,
 'META': 644.68,
 'EQT': 55.59,
 'LUV': 33.64,
 'WELL': 152.63,
 'CME': 286.11,
 'BKNG': 5492.93,
 'EBAY': 72.65,
 'TT': 432.65,
 'WRB': 73.67,
 'WMT': 97.71,
 'RSG': 254.01,
 'TTWO': 225.87}

In [57]:
dfs50["Fiyat"] = dfs50.index.map(fiyat_dict)


In [58]:
dfs50

,0,Yatırım Tutarı,Fiyat
PLTR,145.8450,2000.0,123.01
GEV,61.2600,2000.0,474.43
AXON,49.5400,2000.0,744.96
NRG,44.1575,2000.0,153.82
HWM,43.5100,2000.0,170.09
AVGO,38.3800,2000.0,240.83
TSLA,35.4275,2000.0,360.89
PODD,34.5125,2000.0,323.89
TPR,31.4375,2000.0,78.42
NFLX,29.9550,2000.0,1187.02


In [59]:
dfs50["Alınacak Lot Sayısı"] = dfs50["Yatırım Tutarı"]/dfs50["Fiyat"]

In [60]:
dfs50["Alınacak Lot Sayısı"]=dfs50["Alınacak Lot Sayısı"].round(0).astype(int)

In [61]:
dfs50= dfs50.reset_index()


In [62]:
dfs50.rename(columns={"index": "Hisse Adı"}, inplace=True)

In [63]:
dfs50

,Hisse Adı,0,Yatırım Tutarı,Fiyat,Alınacak Lot Sayısı
0,PLTR,145.8450,2000.0,123.01,16
1,GEV,61.2600,2000.0,474.43,4
2,AXON,49.5400,2000.0,744.96,3
3,NRG,44.1575,2000.0,153.82,13
4,HWM,43.5100,2000.0,170.09,12
5,AVGO,38.3800,2000.0,240.83,8
6,TSLA,35.4275,2000.0,360.89,6
7,PODD,34.5125,2000.0,323.89,6
8,TPR,31.4375,2000.0,78.42,26
9,NFLX,29.9550,2000.0,1187.02,2


In [69]:
dfs50.rename(columns={ 0: "Momentum Skoru"}, inplace=True)

In [70]:
dfs50

,Hisse Adı,Momentum Skoru,Yatırım Tutarı,Fiyat,Alınacak Lot Sayısı
0,PLTR,145.8450,2000.0,123.01,16
1,GEV,61.2600,2000.0,474.43,4
2,AXON,49.5400,2000.0,744.96,3
3,NRG,44.1575,2000.0,153.82,13
4,HWM,43.5100,2000.0,170.09,12
5,AVGO,38.3800,2000.0,240.83,8
6,TSLA,35.4275,2000.0,360.89,6
7,PODD,34.5125,2000.0,323.89,6
8,TPR,31.4375,2000.0,78.42,26
9,NFLX,29.9550,2000.0,1187.02,2


In [74]:
dfs50.to_excel("output.xlsx", index=False, engine='openpyxl')


In [75]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.utils import get_column_letter

wb = load_workbook("output.xlsx")
ws = wb.active

for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        try:
            max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    adjusted_width = max_length + 2
    ws.column_dimensions[col_letter].width = adjusted_width

header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font

wb.save("output.xlsx")